#### Relevant Imports

In [1]:
from sqlalchemy import create_engine, text
import json
from collections import defaultdict
from typing import List, Dict, Any
import csv
from groq import Groq
from google import genai
from google.genai import types
import dotenv
import os

**SQLite DB**  
Let us connect to the SQLite Sample Database what we have  


In [3]:
# There is an engine instance created, which can handle multiple connetions
DB_File = "Sample_2 - Copy.db"

if os.path.exists (DB_File):
    sql_engine = create_engine("sqlite:///"+DB_File)
    conn_1 = sql_engine.connect ()
else:
    print ("DB Files does not exist")

**Get Query**  
Given a user prompt and table schema, LLM can generate SQL query  
The query generation requires specific instructions, so that it can be used correctly

In [ ]:
# Load Env variable and define a Groq Client
dotenv.load_dotenv ()
client_gr = Groq()
# client_gr = Groq (api_key='Your Key')
model_gr = "llama-3.3-70b-versatile"

client_gm = genai.Client()
# client_gm = genai.Client(api_key="Your Key")
model_gm = "gemini-2.5-flash-lite"

In [ ]:
Q_Instr = "From the given SQL table schema, formulate SQL query which answers user's question. \
        Write query to fetch all relevant details to answer the question\
        Respond only the SQL query. No additional text, no fence"

Schema = """
            CREATE TABLE Student_Performance (
            student_id VARCHAR(50),
            age INTEGER,
            gender VARCHAR(50),
            study_hours_per_day REAL,
            social_media_hours REAL,
            netflix_hours REAL,
            part_time_job VARCHAR(50),
            attendance_percentage REAL,
            sleep_hours REAL,
            diet_quality VARCHAR(50),
            exercise_frequency INTEGER,
            parental_education_level VARCHAR(50),
            internet_quality VARCHAR(50),
            mental_health_rating INTEGER,
            extracurricular_participation VARCHAR(50),
            exam_score REAL
          )
        """

Prompt = "On an average students watch netflix for ..."
# Prompt = "Are the students taking enough rest?"
# Prompt = "Is there a correlation between study time and score really?"
# Prompt = "Tell me top 5 scorer who are doing a job additionally, because its quite challenging"

messages=[
    {
        "role": "system",
        "content": Q_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr,
    # temperature=0.0
)

print (completion.choices[0].message.content)

>Additional instructions to fine tune the query generation  
>This will help to get precise output.

In [ ]:
Q_Instr = "From the given SQL table schema, formulate SQL query which answers user's question.\
        Write query to fetch all relevant details to answer the question\
        Respond only the SQL query. No title, no introduction. Give complete query.\
        Its SQLite DB.\
        When searching text field, always convert to lower case and use wild card : %xxx% .\
        If question is not relevant to schema, say 'No relevant data'"


Schema = """
            CREATE TABLE Student_Performance (
            student_id VARCHAR(50),
            age INTEGER,
            gender VARCHAR(50),
            study_hours_per_day REAL,
            social_media_hours REAL,
            netflix_hours REAL,
            part_time_job VARCHAR(50),
            attendance_percentage REAL,
            sleep_hours REAL,
            diet_quality VARCHAR(50),
            exercise_frequency INTEGER,
            parental_education_level VARCHAR(50),
            internet_quality VARCHAR(50),
            mental_health_rating INTEGER,
            extracurricular_participation VARCHAR(50),
            exam_score REAL
          )
        """

Prompt = "Is there a correlation between study time and score really?"
# Prompt = "Tell me top 5 scorers who are doing a job additionally, because its quite challenging"
# Prompt = "What are the kind of diet habits they have?"
# Prompt = "When are the exams?"

messages=[
    {
        "role": "system",
        "content": Q_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr,   

)

Query_String = completion.choices[0].message.content

print (Query_String)

> Try with different model

In [ ]:
# Invoke Gemini LLM API
response = client_gm.models.generate_content(
                model=model_gm,
                # model="gemini-2.5-flash",
                config =types.GenerateContentConfig(
                            system_instruction=Q_Instr,
                            ),
                contents = "Schema :\n"+Schema+"\n Question : \n"+Prompt
)

print("Gemini Response :",response.text)

**Complete Pipeline**
The query string further needs to be used for extraction, provide the context from there and get the answer from LLM  
First invoke of LLM to get the SQL Query. Then next one for getting the response

In [ ]:
# Prompt = "how many students are studying more?"
# Prompt = "Who are the top 5 scorers and how is their social media, netflix habits?"
Prompt = "How much time Students spend on line in general?"
# Prompt = "students whose parents have completed High School, have scored more than 90 in exams"


Q_Instr = "From the given SQL table schema, formulate SQL query which answers user's question.\
        Write query to retrieve required and sufficient information to provide enough context for the question.\
        When searching text field, always convert to lower case and use wild card : %xxx% .\
        Respond only the SQL query. No title, no introduction, no fence. Give complete query.\
        Its SQLite DB.\
        If question is not relevant to schema, say 'No relevant data'"

messages=[
    {
        "role": "system",
        "content": Q_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr,

)

## Get the query string
Query_String = completion.choices[0].message.content
print(Query_String)

#Fetch the data from DB
result = conn_1.execute (text(Query_String))

# Query output into JSON
rows = [row._asdict () for row in result]
json_output = json.dumps(rows,  indent=2)
# print (json_output)

R_Instr = "Using the context given, provide response to the user question or statement.\
            Context is provided as JSON information.\
            Provide a precise answer"

# R_Instr = "Using the context given, provide response to the user question or statement.\
#             Context is provided as JSON information.\
#             Provide a factual answer with elaboration"

messages=[
    {
        "role": "system",
        "content": R_Instr
    },

    {
        "role": "user",
        "content":"Context : \n"+ json_output + "Query : \n" + Prompt
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr,
)

print (completion.choices[0].message.content)

**Check Query**  
SQL query generated might be different each time, depending on the query clarity and complexity  
However, as long as there is a clear schema and right level of information in prompt the generated query would get required information  
As an additional check, there can be an evaluator introduced to check if the data being retrieved by the SQL query is sufficient or not

In [ ]:
# Instruction to Generate SQL query
Q_Instr = "From the given SQL table schema, formulate SQL query which answers user's question.\
        Write query to retrieve all information to provide enough context for the question.\
        When searching text field, always convert to lower case and use wild card : %xxx% .\
        Respond only the SQL query. No title, no introduction. Give complete query.\
        Its SQLite DB.\
        If question is not relevant to schema, say 'No relevant data'"

# Instruction to Check the Query
C_Instr = "You are given a SQL schema, user question and SQL query.\
          I want you to check if the SQL query can fetch enough information from the database to answer question.\
          Give me your **Step evaluation** and **Result** in JSON as 2 fields. No additional response\
          Steps: \
          * Understand the schema.\
          * Understand user question.\
          * Check if SQL query retrives relevant information from the schema to answer the question.\
          \
          Finally, Say 'Pass' / 'Fail'"

# Schema
Schema = """
            CREATE TABLE Student_Performance (
            student_id VARCHAR(50),
            age INTEGER,
            gender VARCHAR(50),
            study_hours_per_day REAL,
            social_media_hours REAL,
            netflix_hours REAL,
            part_time_job VARCHAR(50),
            attendance_percentage REAL,
            sleep_hours REAL,
            diet_quality VARCHAR(50),
            exercise_frequency INTEGER,
            parental_education_level VARCHAR(50),
            internet_quality VARCHAR(50),
            mental_health_rating INTEGER,
            extracurricular_participation VARCHAR(50),
            exam_score REAL
          )
        """

In [ ]:
# Instruction to Generate Wrong SQL query for test
Q_Instr = "From the given SQL table schema, formulate SQL query which answers user's question.\
        Write query to retrieve all information to provide enough context for the question.\
        Respond only the SQL query. No title, no introduction. No fence.\
        While generating query, make one **Deliberate** mistake, that I can use for testing"

In [ ]:
# Prompt = "How much time Students spend on line in general?"
Prompt = "how many students scored marks below 50 ?"
# Prompt = "What do you think the students with high score do differently ?"
# Prompt = "Do students tend to work hard only when they have project?"

# Make use of LLM model to generate SQL query
messages=[
    {
        "role": "system",
        "content": Q_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr,   

)

Query_String = completion.choices[0].message.content

print ("Generated Query :\n", Query_String)

# Call LLM again to check if the Query is sufficient
messages=[
    {
        "role": "system",
        "content": C_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt+ "\n SQL Query : \n" + Query_String
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr,
)

# Check_Status = json.loads(completion.choices[0].message.content)
Check_Status = completion.choices[0].message.content

print (Check_Status)

**Complex Schema**  
When complex schema with multiple tables are there, the query generation would become more subjective  
In this scenario, the evaluation becomes more relevant

In [ ]:
# Instruction to Generate SQL query
Q_Instr = "From the given SQL table schema, formulate SQL query which answers user's question.\
        Write query to retrieve all information to provide enough context for the question.\
        When searching text field, always convert to lower case and use wild card : %xxx% .\
        Respond only the SQL query. No title, no introduction. Give complete query.\
        Its SQLite DB.\
        If question is not relevant to schema, say 'No relevant data'"

# Instruction to Check the Query
C_Instr = "You are given a SQL schema, user question and SQL query.\
          I want you to check if the SQL query can fetch enough information from the database to answer question.\
          Give me your **Step evaluation** and **Result** in JSON as 2 fields. No additional response\
          Steps: \
          * Understand the schema.\
          * Understand user question.\
          * Check if SQL query retrives relevant information from the schema to answer the question.\
          * Check if the Query is accurate in identifying the right relation between tables and fields\
          \
          Finally, Say 'Pass' / 'Fail'"

# Schema
Schema = """
CREATE TABLE [Invoice]
(
    [InvoiceId] INTEGER  NOT NULL,
    [CustomerId] INTEGER  NOT NULL,
    [InvoiceDate] DATETIME  NOT NULL,
    [BillingAddress] NVARCHAR(70),
    [BillingCity] NVARCHAR(40),
    [BillingState] NVARCHAR(40),
    [BillingCountry] NVARCHAR(40),
    [BillingPostalCode] NVARCHAR(10),
    [Total] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Invoice] PRIMARY KEY  ([InvoiceId]),
    FOREIGN KEY ([CustomerId]) REFERENCES [Customer] ([CustomerId]) 		
);

CREATE TABLE [Customer]
(
    [CustomerId] INTEGER  NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    CONSTRAINT [PK_Customer] PRIMARY KEY  ([CustomerId]),
    FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] ([EmployeeId]) 
);

CREATE TABLE [InvoiceLine]
(
    [InvoiceLineId] INTEGER  NOT NULL,
    [InvoiceId] INTEGER  NOT NULL,
    [TrackId] INTEGER  NOT NULL,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    [Quantity] INTEGER  NOT NULL,
    CONSTRAINT [PK_InvoiceLine] PRIMARY KEY  ([InvoiceLineId]),
    FOREIGN KEY ([InvoiceId]) REFERENCES [Invoice] ([InvoiceId]), 		
    FOREIGN KEY ([TrackId]) REFERENCES [Track] ([TrackId]) 		
);
        """

In [ ]:
Prompt = "Top 5 customers by the total value from their invoice"
# Prompt = "The customer with highest total invoice value, purchased which track the most number of units?"

# Make use of LLM model to generate SQL query
messages=[
    {
        "role": "system",
        "content": Q_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr,   

)

Query_String = completion.choices[0].message.content

print ("Generated Query :\n", Query_String)

# Call LLM again to check if the Query is sufficient
messages=[
    {
        "role": "system",
        "content": C_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt+ "\n SQL Query : \n" + Query_String
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr
    # temperature=0.0

)

# Check_Status = json.loads(completion.choices[0].message.content)
Check_Status = completion.choices[0].message.content

print (Check_Status)